# Model Denemeleri

### Önce HuggingFace'ten, hali hazırda tamamen türkçe veriler üzerinde eğitilmiş BerTurk modeli denenecek

In [7]:
from transformers import AutoTokenizer, BertForPreTraining
import torch

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = BertForPreTraining.from_pretrained("dbmdz/bert-base-turkish-cased")

In [2]:
sd_hf = model.state_dict()

for k, v in sd_hf.items():
    print(k, v.shape)

bert.embeddings.word_embeddings.weight torch.Size([32000, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder

In [3]:
torch.equal(sd_hf["bert.embeddings.word_embeddings.weight"], sd_hf["cls.predictions.decoder.weight"]) # tying kontrolü

True

In [4]:
sd_hf["bert.embeddings.word_embeddings.weight"].data_ptr() == sd_hf["cls.predictions.decoder.weight"].data_ptr()  # tying kontrolü

True

In [5]:
torch.equal(sd_hf["cls.predictions.decoder.bias"], sd_hf["cls.predictions.bias"]) # tying kontrolü

True

In [6]:
sd_hf["cls.predictions.decoder.bias"].data_ptr() == sd_hf["cls.predictions.bias"].data_ptr()  # tying kontrolü

True

In [7]:
inputs = tokenizer("Selamlar efendim", return_tensors="pt")
outputs = model(**inputs)

In [8]:
outputs.prediction_logits.shape, outputs.seq_relationship_logits.shape

(torch.Size([1, 4, 32000]), torch.Size([1, 2]))

In [9]:
from transformers import pipeline

fill_masker = pipeline(task="fill-mask", model="dbmdz/bert-base-turkish-cased")
fill_masker("Merhaba [MASK] efendi nasıl gidiyor?")

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'score': 0.05243540555238724,
  'token': 3944,
  'token_str': 'Mehmet',
  'sequence': 'Merhaba Mehmet efendi nasıl gidiyor?'},
 {'score': 0.03593067452311516,
  'token': 7709,
  'token_str': 'hanım',
  'sequence': 'Merhaba hanım efendi nasıl gidiyor?'},
 {'score': 0.03291669860482216,
  'token': 3024,
  'token_str': 'bey',
  'sequence': 'Merhaba bey efendi nasıl gidiyor?'},
 {'score': 0.031684163957834244,
  'token': 7983,
  'token_str': 'hoca',
  'sequence': 'Merhaba hoca efendi nasıl gidiyor?'},
 {'score': 0.030081957578659058,
  'token': 10997,
  'token_str': 'Salih',
  'sequence': 'Merhaba Salih efendi nasıl gidiyor?'}]

In [10]:
from model import BertForPreTraining, BertConfig

%load_ext autoreload
%autoreload 2

In [11]:
default_config = BertConfig()
custom_model = BertForPreTraining(default_config)

In [12]:
custom_sd = custom_model.state_dict()

assert len(custom_sd) == len(sd_hf)

for custom, hf in zip(custom_sd.items(), sd_hf.items()):
    k, v = custom
    k_hf, v_hf = hf
    assert k == k_hf, f"{k} : {k_hf}"
    assert v.shape == v_hf.shape, f"{k} : {v.shape} != {v_hf.shape}"
    print(k, v.shape, "--------", k_hf, v_hf.shape)

bert.embeddings.word_embeddings.weight torch.Size([32000, 768]) -------- bert.embeddings.word_embeddings.weight torch.Size([32000, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768]) -------- bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768]) -------- bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768]) -------- bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768]) -------- bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768]) -------- bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768]) -------- bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768]) -

In [13]:
model_from_hf = BertForPreTraining.from_pretrained()
mm = model_from_hf.state_dict()

loading weights from pretrained bert: dbmdz/bert-base-turkish-cased


In [14]:
from transformers import pipeline

torch.random.manual_seed(42)
fill_masker = pipeline(task="fill-mask", model="dbmdz/bert-base-turkish-cased")
fill_masker("Merhaba [MASK] efendi nasıl gidiyor?") 

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'score': 0.05243540555238724,
  'token': 3944,
  'token_str': 'Mehmet',
  'sequence': 'Merhaba Mehmet efendi nasıl gidiyor?'},
 {'score': 0.03593067452311516,
  'token': 7709,
  'token_str': 'hanım',
  'sequence': 'Merhaba hanım efendi nasıl gidiyor?'},
 {'score': 0.03291669860482216,
  'token': 3024,
  'token_str': 'bey',
  'sequence': 'Merhaba bey efendi nasıl gidiyor?'},
 {'score': 0.031684163957834244,
  'token': 7983,
  'token_str': 'hoca',
  'sequence': 'Merhaba hoca efendi nasıl gidiyor?'},
 {'score': 0.030081957578659058,
  'token': 10997,
  'token_str': 'Salih',
  'sequence': 'Merhaba Salih efendi nasıl gidiyor?'}]

In [15]:
fill_masker("ışınlanma teknolojisini [MASK] Can  bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı") 

[{'score': 0.9745508432388306,
  'token': 22777,
  'token_str': 'Muhammet',
  'sequence': 'ışınlanma teknolojisini Muhammet Can bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı'},
 {'score': 0.016110757365822792,
  'token': 7397,
  'token_str': 'Muhammed',
  'sequence': 'ışınlanma teknolojisini Muhammed Can bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı'},
 {'score': 0.0018351072212681174,
  'token': 3944,
  'token_str': 'Mehmet',
  'sequence': 'ışınlanma teknolojisini Mehmet Can bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı'},
 {'score': 0.00045096001122146845,
  'token': 6001,
  'token_str': 'Hasan',
  'sequence': 'ışınlanma teknolojisini Hasan Can bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı'},
 {'score': 0.0004167473816778511,
  'token': 6391,
  'token_str': 'Cem',
  'sequence': 'ışınlanma teknolojisini Cem Can bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı'}]

In [8]:
# auto-detect device
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"[INFO] using device: {device}")

[INFO] using device: cuda


In [19]:
# WITHOUT PAD TOKEN
from model import FillMaskPipeline, IsNextPipeline
torch.random.manual_seed(42)
model_from_hf.to(device)
fill_mask = FillMaskPipeline(model=model_from_hf, tokenizer=tokenizer, strategy="greedy")
fill_mask(["Merhaba [MASK] efendi nasıl gidiyor?", "[MASK] yaptın?"])

Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['Mehmet', 'doktor', 'Salih', 'hoca', 'hanım'], 'score': ['0.0415', '0.0300', '0.0276', '0.0280', '0.0369']}
Text: [MASK] yaptın? ----> Top 5 Predictions: {'token_str': ['ne', 'neler', 'nasıl', 'neden', 'Ne'], 'score': ['0.5882', '0.0835', '0.0642', '0.0283', '0.0223']}


In [18]:
# WITH PAD TOKEN
torch.random.manual_seed(42)
fill_mask = FillMaskPipeline(model=model_from_hf, tokenizer=tokenizer, strategy="greedy")
fill_mask(["Merhaba [MASK] efendi nasıl gidiyor?", "ışınlanma teknolojisini [MASK] Can  bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı"])

Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['Mehmet', 'doktor', 'Salih', 'hoca', 'hanım'], 'score': ['0.0415', '0.0300', '0.0276', '0.0280', '0.0369']}
Text: ışınlanma teknolojisini [MASK] Can  bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı ----> Top 5 Predictions: {'token_str': ['Muhammet', 'Muhammed', 'Mehmet', 'Hasan', 'Cem'], 'score': ['0.9561', '0.0241', '0.0035', '0.0011', '0.0007']}


In [19]:
# WITH PAD TOKEN
torch.random.manual_seed(42)
fill_mask = FillMaskPipeline(model=model_from_hf, tokenizer=tokenizer, strategy="greedy")
fill_mask(["Merhaba [MASK] efendi nasıl gidiyor?", "ışınlanma teknolojisini [MASK] Can  bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı"])

Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['Mehmet', 'doktor', 'Salih', 'hoca', 'hanım'], 'score': ['0.0415', '0.0300', '0.0276', '0.0280', '0.0369']}
Text: ışınlanma teknolojisini [MASK] Can  bulmuştur. Muhammet Can bu buluş ile büyük alkış topladı ----> Top 5 Predictions: {'token_str': ['Muhammet', 'Muhammed', 'Mehmet', 'Hasan', 'Cem'], 'score': ['0.9561', '0.0241', '0.0035', '0.0011', '0.0007']}


In [20]:
torch.random.manual_seed(42)
fill_mask = FillMaskPipeline(model=model_from_hf, tokenizer=tokenizer, strategy="greedy")
fill_mask(["Merhaba [MASK] efendi nasıl gidiyor?"])

Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['Mehmet', 'doktor', 'Salih', 'hoca', 'hanım'], 'score': ['0.0415', '0.0300', '0.0276', '0.0280', '0.0369']}


In [21]:
fill_mask = FillMaskPipeline(model=model_from_hf, tokenizer=tokenizer, strategy="multinomial")
fill_mask(["Merhaba [MASK] efendi nasıl gidiyor?", "ışınlanma teknolojisi [MASK] gitmektedir."])

Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['Mehmet', 'Mustafa', 'Bey', 'Ali', 'osman'], 'score': ['0.0415', '0.0253', '0.0073', '0.0242', '0.0032']}
Text: ışınlanma teknolojisi [MASK] gitmektedir. ----> Top 5 Predictions: {'token_str': ['yavaş', 'hoşuma', 'ileri', 'yanlış', 'hoşuna'], 'score': ['0.0113', '0.0057', '0.0342', '0.0133', '0.0210']}


In [22]:
# random initialized, so garbage output expected
fill_mask_custom = FillMaskPipeline(model=BertForPreTraining(BertConfig()), tokenizer=tokenizer, strategy="greedy")    
fill_mask_custom(["Merhaba [MASK] efendi nasıl gidiyor?", "ışınlanma teknolojisi [MASK] gitmektedir."]) 

Text: Merhaba [MASK] efendi nasıl gidiyor? ----> Top 5 Predictions: {'token_str': ['istiyordum', 'mesafesinde', '##D', 'Kit', '##ırsın'], 'score': ['0.0002', '0.0002', '0.0003', '0.0003', '0.0002']}
Text: ışınlanma teknolojisi [MASK] gitmektedir. ----> Top 5 Predictions: {'token_str': ['Ster', 'Hon', '##idge', 'mesafesinde', 'Kan'], 'score': ['0.0002', '0.0003', '0.0002', '0.0003', '0.0003']}


In [23]:
nsp_pipeline = IsNextPipeline(model=model_from_hf, tokenizer=tokenizer)

In [24]:
nsp_pipeline([["Çocuk sahibi çift sayısında inanılmaz bir artış var.", "Uzaya ilk Bekiroğ Reis çıkmıştır"],
              ["Bu nsp olayı bir garip oldu. Sanki kablolar ters bağlandı.", "Bir şekilde kabloları ayarlamak gerekli"],
              ["çoğunuz yaş itibariyle tanımaz ama istanbul'un en iyi belediye başkanı justinianus'tur.", "Samsun Ayvacık belediyesi ne yapmak nereye varmak istemektedir!"] ])

Text: ['Çocuk sahibi çift sayısında inanılmaz bir artış var.', 'Uzaya ilk Bekiroğ Reis çıkmıştır'] Predictions ----> isNext: 0.072, notNext: 0.928
Text: ['Bu nsp olayı bir garip oldu. Sanki kablolar ters bağlandı.', 'Bir şekilde kabloları ayarlamak gerekli'] Predictions ----> isNext: 1.000, notNext: 0.000
Text: ["çoğunuz yaş itibariyle tanımaz ama istanbul'un en iyi belediye başkanı justinianus'tur.", 'Samsun Ayvacık belediyesi ne yapmak nereye varmak istemektedir!'] Predictions ----> isNext: 0.056, notNext: 0.944


In [1]:
from bert_implementation_tr.data.data_aux import load_xy_shard, visualize_sample, ModelInput, VisualizeModelInput, get_tokenizer
from model import *
import torch

# auto-detect device
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
print(f"[INFO] using device: {device}")

%load_ext autoreload
%autoreload 2

[INFO] using device: cuda


In [2]:
# device = "cpu" # let's override it right now

In [3]:
temp = load_xy_shard(0, block_size=256)[:50]

In [4]:
temp.shape, 256 * 4

((50, 1024), 1024)

In [5]:
temp.dtype

dtype('uint16')

In [6]:
model_inputs_shard_0 = ModelInput.from_numpy_to_tensors(temp, block_size=256, device=device)


In [7]:
#one_sample_model_input = ModelInput(
#                             input_ids=model_inputs_shard_0.input_ids[0],
#                             labels=model_inputs_shard_0.labels[0],
#                             attention_mask=model_inputs_shard_0.attention_mask[0],
#                             token_type_ids=model_inputs_shard_0.token_type_ids[0],
#                             next_sentence_label=model_inputs_shard_0.next_sentence_label[0]
#                         )
#visualize_sample(VisualizeModelInput(one_sample_model_input, show_ids=True, show_attention_and_segment=True))

### **Önemli!**: *labels'da [CLS] token'ı posizyonu [PAD] olmalı. Yani [CLS] tokeni üzerinde MLM prediction/loss alınmamalı! veri setini ayarla/güncelle*

In [8]:
temp_dict = ModelInput.from_numpy_to_tensors_dict(temp, block_size=256, device=device)

In [9]:
temp_dict["input_ids"].shape

torch.Size([50, 256])

In [10]:
temp_dict["next_sentence_label"]

tensor([0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0,
        1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        1, 1], device='cuda:0')

In [11]:
random_model = BertForPreTraining(BertConfig())
random_model = random_model.to(device)

In [12]:
out = random_model(**temp_dict)

In [13]:
print("loss: ", out.loss.item()) 
print("seq_relationship_logits: ", out.seq_relationship_logits) 
print("prediction_logits: ", out.prediction_logits)

loss:  11.45556354522705
seq_relationship_logits:  tensor([[ 0.2434, -0.3011],
        [ 0.1459, -0.0522],
        [ 0.4600, -0.2076],
        [ 0.2978, -0.3369],
        [ 0.2991, -0.2972],
        [ 0.2087, -0.0824],
        [ 0.3518, -0.4013],
        [ 0.2521, -0.3188],
        [ 0.1899, -0.3676],
        [ 0.3527, -0.0992],
        [ 0.1853, -0.2045],
        [ 0.2412, -0.1185],
        [ 0.3140, -0.1272],
        [ 0.0465, -0.1489],
        [ 0.2862, -0.4020],
        [ 0.3126, -0.2331],
        [ 0.1902, -0.0569],
        [ 0.1527, -0.2361],
        [ 0.2628, -0.3270],
        [ 0.1747, -0.2253],
        [ 0.1054, -0.1326],
        [ 0.4533, -0.1366],
        [ 0.3623, -0.2389],
        [ 0.2102, -0.2206],
        [ 0.3818, -0.1704],
        [ 0.1721, -0.2323],
        [ 0.0577, -0.2579],
        [ 0.4666, -0.2516],
        [ 0.1478, -0.4467],
        [ 0.4947, -0.2397],
        [ 0.3371, -0.2696],
        [ 0.0627, -0.1507],
        [ 0.3724, -0.2838],
        [ 0.4003, -0.2452